The purpose of this notebook is to calculate the power produced by a decay chain after the production of a given nuclide.

In [1]:
using Unitful #https://painterqubits.github.io/Unitful.jl/stable/
using Trapz
#quantity * @u_str("unit abbreviation") 
using Symbolics #https://symbolics.juliasymbolics.org/dev/
#cite https://doi.org/10.48550/arXiv.2105.03949
using Latexify
using Test
#1 * @u_str("mA") is 1 milliamp
using CSV, DataFrames
#using Plots
using PlotlyJS
using Printf
using SymPy #https://docs.juliahub.com/SymPy/ 
using PDFIO
using Unzip
using Interpolations
using Downloads
#plotlyjs()
data_dir = ("C:\\Cross-Section-Data\\")
cross_section_dir = data_dir
parent_dir = "C:\\Users\\engin\\Documents\\GitHub\\Energy\\"
endf8_decay_dir = cross_section_dir * "ENDF_Libraries-2\\ENDF-B-VIII.0\\decay\\"
jeff33_decay_dir = cross_section_dir * "ENDF_Libraries-2\\JEFF-3.3\\decay\\"
jendl5_decay_dir = cross_section_dir * "jendl5-dec_upd5\\"

WebIO._IJuliaInit()

"C:\\Cross-Section-Data\\jendl5-dec_upd5\\"

Now I will find the average decay energies for each nuclide. For the datasets I am using, the average decay energies are all given in keV. 

In [2]:
function get_mass_name(nuclide)
    index = 1
    while (tryparse(Int64, string(nuclide[index])) != nothing && index <= length(nuclide))
        index += 1
    end
    mass = tryparse(Int64, nuclide[1:index-1])
    name =  nuclide[index:end]
    name_1 = uppercase(name[1])
    if (length(name) > 1)
        name = name_1 * name[2]
    else 
      name = name_1
    end
    return mass, name
end
element_symbols = ["H", "He", "Li", "Be", "B", "C", "N", "O", "F", "Ne", 
"Na", "Mg", "Al", "Si", "P", "S", "Cl", "Ar", "K", "Ca", "Sc", "Ti", "V", 
"Cr", "Mn", "Fe", "Co", "Ni", "Cu", "Zn", "Ga", "Ge", "As", "Se", "Br", 
"Kr", "Rb", "Sr", "Y", "Zr", "Nb", "Mo", "Tc", "Ru", "Rh", "Pd", "Ag", 
"Cd", "In", "Sn", "Sb", "Te", "I", "Xe", "Cs", "Ba", "La", "Ce", "Pr", 
"Nd", "Pm", "Sm", "Eu", "Gd", "Tb", "Dy", "Ho", "Er", "Tm", "Yb", "Lu", 
"Hf", "Ta", "W", "Re", "Os", "Ir", "Pt", "Au", "Hg", "Tl", "Pb", "Bi", 
"Po", "At", "Rn", "Fr", "Ra", "Ac", "Th", "Pa", "U", "Np", "Pu", "Am", 
"Cm", "Bk", "Cf", "Es", "Fm", "Md", "No", "Lr", "Rf", "Db", "Sg", "Bh", 
"Hs", "Mt", "Ds", "Rg", "Cn", "Nh", "Fl", "Mc", "Lv", "Ts", "Og"];

Rename files in directory to be easier to parse by removing zero-padding.

In [3]:
function unzero_pad(num_str) 
    if (num_str == '0')
        return num_str
    end
    i = 1
    while i < length(num_str) && num_str[i] == '0'
        i += 1
    end
    return num_str[i : end]
end
function rename_files_in_dir(dir, cutoff_index)
    for f in readdir(dir)
        if tryparse(Int64, string(f[1])) == nothing
            mv(dir * f, dir * f[cutoff_index:end])
        end
        if f != join(unzero_pad.(split(f, "-")), "-")
            mv(dir * f, dir * join(unzero_pad.(split(f, "-")), "-"))
        end
    end
end
rename_files_in_dir(endf8_decay_dir, 12)
rename_files_in_dir(jeff33_decay_dir, 12)
rename_files_in_dir(jendl5_decay_dir, 4)

In [4]:
function get_decay_energy_from_file(file_as_arr, search_string)
    line = [l for l in file_as_arr if length(l) > length(search_string)
             && occursin(search_string, l)]
    if (length(line) > 0)
        line = String(line[1])
        line = join(split(line, search_string, keepempty = false))
        return tryparse(Float64, join(split(line, " ", keepempty = false)[1]))
    else 
        return "Energy not found"
    end
end

get_decay_energy_from_file (generic function with 1 method)

This dictionary stores what strings to look to for in the datasets to find the average energy for each decay type, and for each directory. 

In [5]:
dir_search_string_dict = Dict([
    jeff33_decay_dir => Dict(["alpha" => "ALPHA ENERGY                   =",
    "beta-minus" => "MEAN BETA- ENERGY              =","gamma" => "MEAN GAMMA ENERGY              =", 
    "x-ray" => "MEAN X-RAY ENERGY              =", "auger" => "MEAN AUGER ELECTRON ENERGY     ="
    ]),

    endf8_decay_dir => Dict(["alpha" => "Mean Alpha Energy:", 
    "beta-minus" => "Mean B- Energy:", "beta-plus" => "Mean B+ Energy:",
    "gamma" => "Mean Gamma Energy:", "auger" => "Mean CE+Auger Energy:",
    "neutron" => "Mean Neutron Energy:", "proton" => "Mean Neutron Energy:"]),

    jendl5_decay_dir => Dict(["alpha" => "Mean Alpha Energy",
    "beta-minus" => "Mean B- Energy", "gamma" => "Mean Gamma Energy",
    "neutron" => "Mean Neutron Energy", "proton" => "Mean Proton Energy",
    "auger" => "Mean CE+Auger Energy", 
    "beta-plus and electron capture" => "Mean B+/EC Energy"])  
])

Dict{String, Dict{String, String}} with 3 entries:
  "C:\\Cross-Section-Data\… => Dict("auger"=>"Mean CE+Auger Energy", "beta-plus…
  "C:\\Cross-Section-Data\… => Dict("auger"=>"Mean CE+Auger Energy:", "proton"=…
  "C:\\Cross-Section-Data\… => Dict("auger"=>"MEAN AUGER ELECTRON ENERGY     ="…

Each decay route is stored as its own string of the form isotope,decay type,probability,average energy released, daughter isotope. 

For example,

"3H,B-,100%,5.69keV,3He"

 means that 100% of tritium decays are beta-minus decay, and on average the produced beta particles will have a kinetic energy of 5.69 keV.

In [6]:
comprehensive_decay_chains_dir = parent_dir * "ExportedData\\Comprehensive_decay_chains\\"
decay_file_names = readdir(comprehensive_decay_chains_dir)
decay_chain = CSV.read(comprehensive_decay_chains_dir * decay_file_names[200], DataFrame)[!,1]

3-element Vector{String}:
 "113Mo,B-,100%,3430.0keV,0.08sec" ⋯ 220 bytes ⋯ "3571968e23seconds,113In,stable"
 "113Mo,B-N, %,B-N not yet accoun" ⋯ 25 bytes ⋯ " for ,B-N not yet accounted for"
 "113Mo,B-,100%,3430.0keV,0.08sec" ⋯ 63 bytes ⋯ " for ,B-N not yet accounted for"

In [7]:
CSV.read(comprehensive_decay_chains_dir * "238U.csv", DataFrame)[!, 1]

25-element Vector{String}:
 "238U,A,100%,4187.13keV,1.409963" ⋯ 551 bytes ⋯ "11955686.4seconds,206Pb,stable"
 "238U,SF,0.0000545%,SF not yet a" ⋯ 28 bytes ⋯ "d for ,SF not yet accounted for"
 "238U,A,100%,4187.13keV,1.409963" ⋯ 160 bytes ⋯ " for ,SF not yet accounted for"
 "238U,A,100%,4187.13keV,1.409963" ⋯ 161 bytes ⋯ " for ,Mg not yet accounted for"
 "238U,A,100%,4187.13keV,1.409963" ⋯ 218 bytes ⋯ "or ,24NE not yet accounted for"
 "238U,A,100%,4187.13keV,1.409963" ⋯ 210 bytes ⋯ " for ,SF not yet accounted for"
 "238U,A,100%,4187.13keV,1.409963" ⋯ 261 bytes ⋯ "for ,14C not yet accounted for"
 "238U,A,100%,4187.13keV,1.409963" ⋯ 549 bytes ⋯ "11955686.4seconds,206Pb,stable"
 "238U,A,100%,4187.13keV,1.409963" ⋯ 546 bytes ⋯ "11955686.4seconds,206Pb,stable"
 "238U,A,100%,4187.13keV,1.409963" ⋯ 540 bytes ⋯ "11955686.4seconds,206Pb,stable"
 "238U,A,100%,4187.13keV,1.409963" ⋯ 419 bytes ⋯ "for ,B-A not yet accounted for"
 "238U,A,100%,4187.13keV,1.409963" ⋯ 538 bytes ⋯ "11955686.4seconds,206

In [8]:
CSV.read(comprehensive_decay_chains_dir * "238U.csv", DataFrame)[!, 1]

25-element Vector{String}:
 "238U,A,100%,4187.13keV,1.409963" ⋯ 551 bytes ⋯ "11955686.4seconds,206Pb,stable"
 "238U,SF,0.0000545%,SF not yet a" ⋯ 28 bytes ⋯ "d for ,SF not yet accounted for"
 "238U,A,100%,4187.13keV,1.409963" ⋯ 160 bytes ⋯ " for ,SF not yet accounted for"
 "238U,A,100%,4187.13keV,1.409963" ⋯ 161 bytes ⋯ " for ,Mg not yet accounted for"
 "238U,A,100%,4187.13keV,1.409963" ⋯ 218 bytes ⋯ "or ,24NE not yet accounted for"
 "238U,A,100%,4187.13keV,1.409963" ⋯ 210 bytes ⋯ " for ,SF not yet accounted for"
 "238U,A,100%,4187.13keV,1.409963" ⋯ 261 bytes ⋯ "for ,14C not yet accounted for"
 "238U,A,100%,4187.13keV,1.409963" ⋯ 549 bytes ⋯ "11955686.4seconds,206Pb,stable"
 "238U,A,100%,4187.13keV,1.409963" ⋯ 546 bytes ⋯ "11955686.4seconds,206Pb,stable"
 "238U,A,100%,4187.13keV,1.409963" ⋯ 540 bytes ⋯ "11955686.4seconds,206Pb,stable"
 "238U,A,100%,4187.13keV,1.409963" ⋯ 419 bytes ⋯ "for ,B-A not yet accounted for"
 "238U,A,100%,4187.13keV,1.409963" ⋯ 538 bytes ⋯ "11955686.4seconds,206

In [9]:
function parse_decay_chain(decay_chain)
    arr = split(decay_chain, ",")
    decay_steps = Int(floor(length(arr) / 5))
    decays = Vector{Any}(undef, decay_steps + 1)
    for decay_step in 1 : decay_steps
       decays[decay_step] = arr[5 * (decay_step - 1) + 1 : 5*decay_step] 
    end
    decays[decay_steps + 1] = arr[5 * decay_steps + 1 : end]
    return decays
end
parse_decay_chain(decay_chain)

1-element Vector{Any}:
 [["113Mo,B-,100%,3430.0keV,0.08seconds,113Tc,B-,100%,2712.98keV,0.152seconds,113Ru,B-,100%,2531.81keV,0.8seconds,113Rh,B-,100%,1580.16keV,2.8seconds,113Pd,B-,100%,1362.75keV,93seconds,113Ag,B-,100%,762.85keV,19332seconds,113Cd,B-,100%,92.6keV,2.5371768483571968e23seconds,113In,stable", "113Mo,B-N, %,B-N not yet accounted for,B-N not accounted for ,B-N not yet accounted for", "113Mo,B-,100%,3430.0keV,0.08seconds,113Tc,B-N,2.1%,B-N not yet accounted for,B-N not accounted for ,B-N not yet accounted for"]]

In [10]:
decay_dict = Dict([])
for file in decay_file_names
    nuclide = split(file, ".csv")[1]
    file_path = comprehensive_decay_chains_dir * file
    decay_chains = CSV.read(file_path, DataFrame, stringtype = String)[!, 1]
    decay_chains = [parse_decay_chain(decay_chain) for decay_chain in decay_chains]
    decay_dict[nuclide] = decay_chains
end
decay_dict

Dict{Any, Any} with 3089 entries:
  "113Sn" => Vector{Any}[[SubString{String}["113Sn", "EC+B+", "100%", "0.0keV",…
  "243Cf" => Vector{Any}[[SubString{String}["243Cf", "EC+B+", "86%", "EC+B+ not…
  "168Eu" => Vector{Any}[[SubString{String}["168Eu", "B-", "100%", "2168.0keV",…
  "160Dy" => Vector{Any}[[SubString{String}["160Dy", "stable"]]]
  "162Sm" => Vector{Any}[[SubString{String}["162Sm", "B-", "100%", "1383.0keV",…
  "57V"   => Vector{Any}[[SubString{String}["57V", "B-", "100%", "3558.89keV", …
  "208Po" => Vector{Any}[[SubString{String}["208Po", "A", "99.996%", "5114.7keV…
  "174Ir" => Vector{Any}[[SubString{String}["174Ir", "EC", "99.5%", "EC not yet…
  "144Gd" => Vector{Any}[[SubString{String}["144Gd", "EC+B+", "100%", "0.0keV",…
  "80Sr"  => Vector{Any}[[SubString{String}["80Sr", "EC+B+", "100%", "35.78keV"…
  "33Ar"  => Vector{Any}[[SubString{String}["33Ar", "EC+B+", "100%", "3806.93ke…
  "220Pa" => Vector{Any}[[SubString{String}["220Pa", "A", "100%", "Energy not f…
  "108In" 

In [20]:
decay_dict["218At"][1]

7-element Vector{Any}:
 SubString{String}["218At", "A", "99.95%", "6686.02keV", "1.28seconds"]
 SubString{String}["214Bi", "B-", "99.979%", "640.02keV", "1182.6seconds"]
 SubString{String}["214Po", "A", "100%", "7686.73keV", "0.00016346seconds"]
 SubString{String}["210Pb", "B-", "100%", "6.08keV", "700563756.6359425seconds"]
 SubString{String}["210Bi", "B-", "100%", "389.0keV", "433036.8seconds"]
 SubString{String}["210Po", "A", "100%", "5304.38keV", "11955686.4seconds"]
 SubString{String}["206Pb", "stable"]

In [63]:
decay_chains = decay_dict["238Pu"]



LoadError: UndefVarError: max_steps not defined

In [70]:
function make_decay_tree(isotope)
    decay_chains = decay_dict[isotope]
    lengths = [length(chain) for chain in decay_chains]
    max_step = maximum(lengths)
    print(max_step)
    steps = [unique([decay_chain[step][1]
                    for decay_chain in decay_chains
                if length(decay_chain) >= step])
            for step in 1:max_step]
    return [step for step in steps if length(step) > 0]
end
tree = make_decay_tree("238Pu")

13

13-element Vector{Vector{SubString{String}}}:
 ["238Pu"]
 ["234U", "SF not yet accounted for"]
 ["230Th", "SF not yet accounted for", "Mg not yet accounted for"]
 ["226Ra", "24NE not yet accounted for", "SF not yet accounted for"]
 ["222Rn", "14C not yet accounted for"]
 ["218Po"]
 ["214Pb", "218At"]
 ["214Bi", "218Rn"]
 ["214Po", "210Tl", "B-A not yet accounted for"]
 ["210Pb", "B-N not yet accounted for"]
 ["210Bi", "206Hg"]
 ["210Po", "206Tl"]
 ["206Pb"]

In [14]:
function all_decay_modes_accounted_for(decay_chain) 
    for decay_step in decay_chain
        for item in decay_step
            if length(item) > 21
                if occursin("not yet accounted for", item)
                    return false
                end
            end
        end
    end
    return true
end

function get_decay_info(decay_chain)
    if any([any([occursin(" ", element) || occursin("nothing", element) 
             for element in step]) for step in decay_chain])
        return "Insufficient data"
    end
    if all_decay_modes_accounted_for(decay_chain)
        decay_energies = [parse(Float64, decay_step[4][1:end-3]) 
                            for decay_step in decay_chain 
                                if length(decay_step) > 3]
        decay_probabilities = [parse(Float64, decay_step[3][1:end-1]) / 100 
                                for decay_step in decay_chain  
                                    if length(decay_step) > 2] 
        e_folding_times = [parse(Float64, decay_step[5][1:end-7]) / log(2)
                            for decay_step in decay_chain 
                                if length(decay_step) > 3] 
        return decay_probabilities, decay_energies, e_folding_times
    end
    return "Decay mode(s) not yet accounted for"
end
get_decay_info(decay_dict["210Pb"][3])

([1.0, 1.32e-6, 1.0], [6.08, 0.00617, 538.37], [1.0106998575252168e9, 624740.0438824259, 363.73227370892545])

In [15]:
decay_dict["207Bi"]

1-element Vector{Vector{Any}}:
 [SubString{String}["207Bi", "EC+B+", "100%", "0.146keV", "995621014.4983776seconds"], SubString{String}["207Pb", "stable"]]

Which is correct as seen by the cell below :) 

In [16]:
decay_dict["210Pb"][2]

4-element Vector{Any}:
 SubString{String}["210Pb", "A", "0.0000019%", "0.0keV", "700563756.6359425seconds"]
 SubString{String}["206Hg", "B-", "100%", "400.35keV", "499.2seconds"]
 SubString{String}["206Tl", "B-", "100%", "538.37keV", "252.12seconds"]
 SubString{String}["206Pb", "stable"]

However, this only works for some decay chains

In [17]:
get_decay_info(decay_dict["238U"][2])

"Insufficient data"

I derived the decay rate of the n-th generation nuclide in a decay chain here:

https://www.overleaf.com/project/627613def6b848465d85e5bb

https://github.com/MarcosP7635/Math-for-Energy/blob/main/main.tex

file:///C:/Users/engin/Downloads/Math_for_Energy(4).pdf 

In [18]:
factor(t, e_folding_time) = 1 - exp(t / -e_folding_time)

factor (generic function with 1 method)

In [19]:
a_prod(e_folding_times, t) = prod([factor(t, e_folding_times[k])
                                   for k in 2:length(e_folding_times)])

a_prod (generic function with 1 method)

In [20]:
r(e_fold_times, t) = (a_prod(e_fold_times, t) * exp(t / -e_fold_times[1])
                      / e_fold_times[1])


r (generic function with 1 method)

In [21]:
λ_0, λ_1, λ_2, t = @variables λ_0, λ_1, λ_2, t
expected_output = (1 - exp(t / -λ_1)) * (1 - exp(t / -λ_2)) * exp(t / -λ_0) / λ_0

((1 - exp(t / (-λ_1)))*(1 - exp(t / (-λ_2)))*exp(t / (-λ_0))) / λ_0

In [22]:
e_folding_times = [λ_0, λ_1, λ_2]
r(e_folding_times, t)
Test.@test r(e_folding_times, t) - expected_output == 0

Test Passed
  Expression: r(e_folding_times, t) - expected_output == 0
   Evaluated: 0 == 0

It works! :) 

The average beta decay energy is given in keV. 

In [23]:
joules_per_keV = Rational(uconvert(@u_str("J"), 1 * @u_str("keV")) / @u_str("J"))

337//2103388558093277156

Avogadro's number

In [24]:
N_A = 6.02214076e23
conversion_factor = Rational(joules_per_keV * N_A)

6475021031458043//67108864

Now we calculate the power in watts

In [25]:
power_per_mole(decay_rate, decay_energy) = decay_rate * decay_energy * conversion_factor

power_per_mole (generic function with 1 method)

In [26]:
function power_per_mole(decay_probabilities, decay_energies, e_folding_times, t)
    l = length(decay_probabilities)
    if l == 1
        return decay_probabilities .* power_per_mole(r(e_folding_times, t), decay_energies[end])
    end
    output = decay_probabilities[end] * sum(power_per_mole(r(e_folding_times, t),
                                             decay_energies[end]))
    for i in 2:l
        index = l - i + 1
        output = decay_probabilities[index] * (output + 
                    power_per_mole(r(e_folding_times[1:index], t), decay_energies[index]))
    end
    return output
end

power_per_mole (generic function with 2 methods)

In [27]:
E_0, E_1, E_2, p_0, p_1, p_2, t = @variables E_0, E_1, E_2, p_0, p_1, p_2, t
expected_output = conversion_factor * (
    p_0 * (E_0 * exp(t / -λ_0) / λ_0 + 
        p_1 * ((1 - exp(t / -λ_1)) * E_1 * exp(t / -λ_0) / λ_0 + 
            p_2 * (1 - exp(t / -λ_1)) * (1 - exp(t / -λ_2)) * E_2 * exp(t / -λ_0) / λ_0
        )
    )
)          

(6475021031458043//67108864)*p_0*(p_1*((E_1*(1 - exp(t / (-λ_1)))*exp(t / (-λ_0))) / λ_0 + (E_2*p_2*(1 - exp(t / (-λ_1)))*(1 - exp(t / (-λ_2)))*exp(t / (-λ_0))) / λ_0) + (E_0*exp(t / (-λ_0))) / λ_0)

In [28]:
decay_energies = [E_0, E_1, E_2]
decay_probabilities = [p_0, p_1, p_2]
Test.@test Symbolics.simplify(power_per_mole(decay_probabilities,
                                decay_energies, e_folding_times, t) 
            - expected_output) == 0

Test Passed
  Expression: Symbolics.simplify(power_per_mole(decay_probabilities, decay_energies, e_folding_times, t) - expected_output) == 0
   Evaluated: 0 == 0

The function named power_all_decay_modes calculates the power in watts produced by one mole of a given isotope after a given time in seconds. 
Note that so far, this only includes alpha decay, beta decay, neutron decay, and proton decay. 

In [29]:
function power_all_decay_modes(isotope, time)
    decay_chains_given_isotope = decay_dict[isotope]
    power = 0
    for decay_chain in decay_chains_given_isotope
        test = get_decay_info(decay_chain)
        if (test != "Decay mode(s) not yet accounted for" 
            && test[1] != Float64[] && test != "Insufficient data")
            decay_probabilities, decay_energies, e_folding_times = test
            sub_power = power_per_mole(decay_probabilities, decay_energies, 
                                        e_folding_times, time)
            if (typeof(sub_power) == Vector{Float64} || 
                typeof(sub_power) == Vector{BigFloat})
                power += sub_power[1]
            else
                power += sub_power
            end
        end
    end
    return power
end

power_all_decay_modes (generic function with 1 method)

It takes 1 second to calculate the power of every decay chain for a given time value. 

In [30]:
exponents_for_time_array = -10:20/1000:10
collect(exponents_for_time_array)

1001-element Vector{Float64}:
 -10.0
  -9.98
  -9.96
  -9.94
  -9.92
  -9.9
  -9.88
  -9.86
  -9.84
  -9.82
  -9.8
  -9.78
  -9.76
   ⋮
   9.78
   9.8
   9.82
   9.84
   9.86
   9.88
   9.9
   9.92
   9.94
   9.96
   9.98
  10.0

In [31]:
time_array = [BigFloat(10)^x for x in exponents_for_time_array]

1001-element Vector{BigFloat}:
 1.000000000000000000000000000000000000000000000000000000000000000000000000000003e-10
 1.047128548050898505549645783823682271757193893433574137820153812424537637016191e-10
 1.0964781961431828604257317209264159331313714105011702911985913060102831306613e-10
 1.148153621496884066480156540139268838588675605873626387416861363160599339507005e-10
 1.202264434617413102533306847068551134853580025789314084263608408653746937282328e-10
 1.25892541179416618056939287182433126845996218444738646667433027351526766150798e-10
 1.318256738556404729589129642168660706457656431654852798000801526451019756623137e-10
 1.38038426460288664458362703761266589785155477230948719984049693714804016050564e-10
 1.445439770745927984903979184458196052506877843919927508765240153411912508988761e-10
 1.513561248436207171938059987874779303581263483786977823789873399913939914213657e-10
 1.584893192461110892181946192724010648088344291738932269688212001583469968929013e-10
 1.659586907437563078013118

In [32]:
zero_gen_isotopes = [split(file, ".csv")[1] for file in readdir(comprehensive_decay_chains_dir)]

3089-element Vector{SubString{String}}:
 "100Ag"
 "100Cd"
 "100In"
 "100Kr"
 "100Mo"
 "100Nb"
 "100Pd"
 "100Rb"
 "100Rh"
 "100Ru"
 "100Sn"
 "100Sr"
 "100Tc"
 ⋮
 "99Pd"
 "99Rb"
 "99Rh"
 "99Ru"
 "99Sn"
 "99Sr"
 "99Tc"
 "99Y"
 "99Zr"
 "9Be"
 "9C"
 "9Li"

In [33]:
power_time_series = [[power_all_decay_modes(isotope, time)
                        for time in time_array] for isotope in zero_gen_isotopes]; 
power_time_series_dict = Dict([])
stop = length(power_time_series)
for i in 1:stop
        power_time_series_dict[zero_gen_isotopes[i]] = power_time_series[i]
end
power_time_series_dict

Dict{Any, Any} with 3089 entries:
  "113Sn" => BigFloat[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0,…
  "243Cf" => BigFloat[5.79205e+07, 5.79205e+07, 5.79205e+07, 5.79205e+07, 5.792…
  "168Eu" => BigFloat[7.24963e+11, 7.24963e+11, 7.24963e+11, 7.24963e+11, 7.249…
  "160Dy" => [0, 0, 0, 0, 0, 0, 0, 0, 0, 0  …  0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
  "162Sm" => BigFloat[3.85388e+10, 3.85388e+10, 3.85388e+10, 3.85388e+10, 3.853…
  "57V"   => BigFloat[7.43792e+11, 7.43792e+11, 7.43792e+11, 7.43792e+11, 7.437…
  "208Po" => BigFloat[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0,…
  "174Ir" => [0, 0, 0, 0, 0, 0, 0, 0, 0, 0  …  0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
  "144Gd" => BigFloat[0.00348475, 0.00364898, 0.00382095, 0.00400103, 0.0041895…
  "80Sr"  => BigFloat[375183.0, 375183.0, 375183.0, 375183.0, 375183.0, 375183.…
  "33Ar"  => BigFloat[1.47169e+12, 1.47169e+12, 1.47169e+12, 1.47169e+12, 1.471…
  "220Pa" => [0, 0, 0, 0, 0, 0, 0, 0, 0, 0  …  0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
  "108

In [34]:
function plot_power_time_series(nuclide, time_array)
    power_time_series = Float64.(power_time_series_dict[nuclide])
    time_array = Float64.(time_array)
    plot(scatter(x = time_array, y = power_time_series, mode = "markers"), 
    Layout(xaxis_type = "log", yaxis_type = "log"))#, yaxis_type = "log"))
end
plot_power_time_series("207Bi", time_array)

data: [
  "scatter with fields mode, type, x, and y"
]

layout: "layout with fields margin, template, xaxis, and yaxis"

In [35]:
values(power_time_series_dict)

ValueIterator for a Dict{Any, Any} with 3089 entries. Values:
  BigFloat[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, …
  BigFloat[5.792047897302005391336331231777872359229324183944982144919693825975…
  BigFloat[7.249633292054007246720920765094881293122669042286549707840698313624…
  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0  …  0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
  BigFloat[3.853875632674555958658427628817897047161779472341905197275781097056…
  BigFloat[7.437917271028484003105909170080619042933438503858967839184469667931…
  BigFloat[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, …
  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0  …  0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
  BigFloat[0.003484748401770615148704758599019514406799498176008992995549476550…
  BigFloat[375182.5048195058572735873330274084234950009597068444032893047416861…
  BigFloat[1.471687310272385191676159064298314447624894021507074154922761233312…
  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0  …  0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
  BigFloat[

In [36]:
power_matrix = Matrix{Union{Float64, BigFloat, Int64}}(undef, stop, length(time_array))
for row in 1:stop
    power_matrix[row, :] = power_time_series_dict[zero_gen_isotopes[row]]
end
power_df = DataFrame(power_matrix, string.(time_array))

,1.000000000000000000000000000000000000000000000000000000000000000000000000000003e-10
,Union…
1,0
2,0
3,0
4,0
5,0
6,0
7,0
8,0
9,37082.24512356391637563095724115687593677130846943214712681923974415432433154552


In [37]:
power_df[!, "Parent Isotope"] = zero_gen_isotopes

3089-element Vector{SubString{String}}:
 "100Ag"
 "100Cd"
 "100In"
 "100Kr"
 "100Mo"
 "100Nb"
 "100Pd"
 "100Rb"
 "100Rh"
 "100Ru"
 "100Sn"
 "100Sr"
 "100Tc"
 ⋮
 "99Pd"
 "99Rb"
 "99Rh"
 "99Ru"
 "99Sn"
 "99Sr"
 "99Tc"
 "99Y"
 "99Zr"
 "9Be"
 "9C"
 "9Li"

In [38]:
power_df[2, "Parent Isotope"]

"100Cd"

In [39]:
CSV.write(cross_section_dir * "All_decay_modes_power_time_series_1001_steps.csv", power_df)

"C:\\Cross-Section-Data\\All_decay_modes_power_time_series_1001_steps.csv"

https://drive.google.com/file/d/1aOk0XagAuHJ9FUSMbESXMMc5aSb0LOwH/view?usp=sharing

Now to find the highest power isotopes as a function of time. 

In [40]:
str_time_array = string.(time_array)
max_power_isotopes = [zero_gen_isotopes[argmax(power_df[!, t])]
                    for t in str_time_array]
max_power_values = [power_df[argmax(power_df[!, t]), t] for t in str_time_array]

1001-element Vector{BigFloat}:
   5.295743860585541362268520139389228751761384653291691858677615325622021706240465e+18
   5.295602062348459793778355869946590163913540751252128432599910764913697003097624e+18
   5.2954535854357413400960358241595519371319396397605322985262900764785790274447e+18
   5.295298115483582815624013070332802795608535540072884556144918751590509940466318e+18
   5.295135323350518932997052117193964630385357369801128213507381514388764886517328e+18
   5.294964864424621168033657609865008490047222743181412603294687127797452666545276e+18
   5.29478637789839766844490455838341702186232761897512964482492203052119369251536e+18
   5.294599486009905637335299463157407362427717166731729046682470408063784792444796e+18
   5.294403793248521142147659127270222552008139107188154852798499356631053365950582e+18
   5.294198885523741128040869575977176064330715275602561635903276762565998785808091e+18
   5.293984329295320343119224724409779775614548824082189372740317516423370074307745e+18
   5

In [41]:
max_power_df = DataFrame(Time_seconds = time_array, Power_watts = max_power_values,
Isotope = max_power_isotopes, stringtype = string)

,Time_seconds
,BigFloat
1,1.000000000000000000000000000000000000000000000000000000000000000000000000000003e-10
2,1.047128548050898505549645783823682271757193893433574137820153812424537637016191e-10
3,1.0964781961431828604257317209264159331313714105011702911985913060102831306613e-10
4,1.148153621496884066480156540139268838588675605873626387416861363160599339507005e-10
5,1.202264434617413102533306847068551134853580025789314084263608408653746937282328e-10
6,1.25892541179416618056939287182433126845996218444738646667433027351526766150798e-10
7,1.318256738556404729589129642168660706457656431654852798000801526451019756623137e-10
8,1.38038426460288664458362703761266589785155477230948719984049693714804016050564e-10
9,1.445439770745927984903979184458196052506877843919927508765240153411912508988761e-10


In [42]:
unique(max_power_df[!,"Isotope"])

27-element Vector{SubString{String}}:
 "218Th"
 "214At"
 "271Ds"
 "156Hf"
 "218Rn"
 "160W"
 "259Rf"
 "259Lr"
 "269Hs"
 "263Db"
 "261Rf"
 "226Pa"
 "223Ac"
 ⋮
 "259No"
 "254Fm"
 "255Fm"
 "222Rn"
 "255Es"
 "258Md"
 "257Fm"
 "254Es"
 "228Th"
 "243Cm"
 "238Pu"
 "251Cf"

In [43]:
plot(scatter(max_power_df, x=:Time_seconds, y=:Power_watts, mode="markers",
text=:Isotope, color=:Isotope),
Layout(xaxis_type = "log", yaxis_type = "log", show_scale = true))

data: [
  "scatter with fields color, mode, text, type, x, and y"
]

layout: "layout with fields margin, show, template, xaxis, and yaxis"

In [44]:
one_second_index = [x for x in 1:length(time_array) if time_array[x] == 1][1]
time_array[one_second_index]

1.0

In [45]:
x = Float64.(time_array)
sub_second_energy_approx_dict = Dict([])
for key in keys(power_time_series_dict)
    sub_second_energy_approx_dict[key] = trapz(x[1:one_second_index], 
                            power_time_series_dict[key][1:one_second_index])
end
maximum(values(sub_second_energy_approx_dict))

3.311592416320192083122218784804065460426074682776981855667390223271079758392337e+13

In [46]:
sub_second_approx_energy_df = DataFrame( 
    Isotopes = [String(key) for key in keys(sub_second_energy_approx_dict)],
    Energy_Joules = [Float64(energy) for energy 
        in values(sub_second_energy_approx_dict)],
                    )

,Isotopes,Energy_Joules
,String,Float64
1,113Sn,0.0
2,243Cf,5.78892e7
3,168Eu,2.10731e11
4,160Dy,0.0
5,162Sm,3.39567e10
6,57V,3.0587e11
7,208Po,0.0
8,174Ir,0.0
9,144Gd,1.70063e7


In [47]:
plot(scatter(sub_second_approx_energy_df, y=:Energy_Joules, mode="markers",
            text=:Isotopes, color=:Isotopes),
Layout(xaxis_type = "log", yaxis_type = "log", show_scale = true))

data: [
  "scatter with fields color, mode, text, type, and y"
]

layout: "layout with fields margin, show, template, xaxis, and yaxis"

In [48]:
function robust_log(x)
    if x > 0
        return log(x) / log(10)
    else
        return nothing
    end    
end
sub_second_approx_energy_df[!, "Log10_Energy_Joules"] = [robust_log(energy) for energy in 
                                            sub_second_approx_energy_df[!, "Energy_Joules"]]

3089-element Vector{Union{Nothing, Float64}}:
   nothing
  7.76259771864233
 11.323728715772386
   nothing
 10.530925281962762
 11.485537419236119
   nothing
   nothing
  7.230609674702864
  5.774783603592845
 11.571263997266403
   nothing
  6.451719565917122
  ⋮
   nothing
  9.84279962104166
   nothing
  9.495638672726834
   nothing
 11.586960369087345
   nothing
   nothing
   nothing
  4.169514551273601
   nothing
   nothing

In [49]:
plot(sub_second_approx_energy_df, x=:Log10_Energy_Joules, kind = "histogram", nbinsx = 100, 
Layout(yaxis_type = "log"))

data: [
  "histogram with fields nbinsx, type, x, xaxis, and yaxis"
]

layout: "layout with fields legend, margin, template, xaxis, and yaxis"

There are over 100 isotopes that produce over 1 TJ (10^12 Joules) within 100 years of their decay :)

In [50]:
Joule_per_MeV = uconvert(@u_str("J"), 1 * @u_str("MeV")) / @u_str("J")

1.6021766339999998e-13

In [51]:
min_MeV_per_neutron = 15.957423529411766
min_Joules_per_mol_neutron = N_A * min_MeV_per_neutron * Joule_per_MeV

1.539657309067616e12

In [52]:
log(2 * min_Joules_per_mol_neutron)/log(10)

12.488454063680592

In [53]:
potential_isotopes_rows = [row for row in 1:size(sub_second_approx_energy_df)[1]
        if sub_second_approx_energy_df[row, "Energy_Joules"] > min_Joules_per_mol_neutron / .5]
sub_second_approx_energy_df[potential_isotopes_rows, :]

,Isotopes,Energy_Joules,Log10_Energy_Joules
,String,Float64,Union…
1,218Rn,4.27907e12,12.6313
2,156Hf,3.40641e12,12.5323
3,271Ds,3.31159e13,13.52
4,259Rf,5.77608e12,12.7616
5,218Fr,3.21476e12,12.5071
6,160W,4.99594e12,12.6986
7,263Sg,3.88664e12,12.5896


In [54]:
CSV.write(parent_dir * "ExportedData\\net_sub_second_energy_isotopes_betavoltaics.csv",
            sub_second_approx_energy_df[potential_isotopes_rows, :])

"C:\\Users\\engin\\Documents\\GitHub\\Energy\\ExportedData\\net_sub_second_energy_isotopes_betavoltaics.csv"

In [55]:
decay_dict["207Bi"]

1-element Vector{Vector{Any}}:
 [SubString{String}["207Bi", "EC+B+", "100%", "0.146keV", "995621014.4983776seconds"], SubString{String}["207Pb", "stable"]]